In [ ]:
####-----------------------------------------

# Imports
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc, Input, Output, State
import numpy as np
import dash_bootstrap_components as dbc
import dash_daq as daq
from simulation.functions import runSimulation, aggregateOutputs, readData
import os
import sys

####-----------------------------------------

# Datasets
datasets = readData(area="kamppi")
dataset_names = list(datasets.keys())
max_sim_minute = datasets["Baseline emissions"]["Simulation timestep"].max()/60
timeline = np.arange(1, max_sim_minute+1, 1)

####-----------------------------------------

# Interface text variables
mobility_modes = ["Pedestrians", "Private cars", "Bicycles", "Buses", "Trams"]
objectives = ["Summary", "Air quality", "Livability", "Traffic"]
seasons = ["Summer", "Winter"]
weekdays = ["Weekday", "Weekend"]
mobility_demand = ["Low", "Regular", "High"]
areas = ["Arabia", "Jätkäsaari", "Kamppi", "Vihdintie"]
timeline_functions = {"Average": "mean", "Sum": "sum"}

####-----------------------------------------

# From variable-situation-to-dataset-columns mapping for filtering

# (Situation, variable) -> (dataset, [list of columns])
from_situ_var_to_data_cols = {
    ("Baseline", "Trips"): (
        "Baseline trips",
        ["Travel time", "Lost time", "Mobility mode", "Mobility flow",],
    ),
    ("Baseline", "Mobility flow"): (
        "Baseline emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Mobility flow",
            "Mobility mode",
        ],
    ),
    ("Baseline", "Speed"): (
        "Baseline emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Mobility flow",
            "Mobility mode",
            "Speed",
        ],
    ),
    ("Baseline", "Travel time"): (
        "Baseline trips",
        ["Mobility mode", "Travel time", "Mobility flow",],
    ),
    ("Baseline", "Lost time"): (
        "Baseline trips",
        ["Mobility mode", "Lost time", "Mobility flow",],
    ),
    ("Baseline", "Noise"): (
        "Baseline noise",
        ["Name", "Edge", "Simulation timestep", "Longitude", "Latitude", "Noise", "Mobility flow"],
    ),
    ("Optimized", "Mobility flow"): (
        "Optimized emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Mobility flow",
            "Mobility mode",
        ],
    ),
    ("Optimized", "Speed"): (
        "Optimized emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Mobility flow",
            "Mobility mode",
            "Speed",
        ],
    ),
    ("Optimized", "Travel time"): (
        "Optimized trips",
        ["Mobility mode", "Travel time", "Mobility flow",],
    ),
    ("Optimized", "Noise"): (
        "Optimized noise",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Noise",
            "Mobility flow",
        ],
    ),
    ("Baseline", "Carbon monoxide"): (
        "Baseline emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Carbon monoxide",
            "Mobility mode",
            "Mobility flow",
        ],
    ),
    ("Baseline", "Carbon dioxide"): (
        "Baseline emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Carbon dioxide",
            "Mobility mode",
            "Mobility flow",
        ],
    ),
    ("Baseline", "Hydrocarbon"): (
        "Baseline emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Hydrocarbon",
            "Mobility mode",
            "Mobility flow",
        ],
    ),
    ("Baseline", "Nitrogen oxides"): (
        "Baseline emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Nitrogen oxides",
            "Mobility mode",
            "Mobility flow",
        ],
    ),
    ("Baseline", "Respirable particles"): (
        "Baseline emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Respirable particles",
            "Mobility mode",
            "Mobility flow",
        ],
    ),
    ("Baseline", "Fine particles"): (
        "Baseline emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Fine particles",
            "Mobility mode",
            "Mobility flow",
        ],
    ),
    ("Optimized", "Carbon monoxide"): (
        "Optimized emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Carbon monoxide",
            "Mobility mode",
            "Mobility flow",
        ],
    ),
    ("Optimized", "Carbon dioxide"): (
        "Optimized emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Carbon dioxide",
            "Mobility mode",
            "Mobility flow",
        ],
    ),
    ("Optimized", "Hydrocarbon"): (
        "Optimized emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Hydrocarbon",
            "Mobility mode",
            "Mobility flow",
        ],
    ),
    ("Optimized", "Nitrogen oxides"): (
        "Optimized emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Nitrogen oxides",
            "Mobility mode",
            "Mobility flow",
        ],
    ),
    ("Optimized", "Respirable particles"): (
        "Optimized emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Respirable particles",
            "Mobility mode",
            "Mobility flow",
        ],
    ),
    ("Optimized", "Fine particles"): (
        "Optimized emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Fine particles",
            "Mobility mode",
            "Mobility flow",
        ],
    ),
    ("Baseline", "Relocation rate"): (
        "Baseline livability",
        ["Name", "Longitude", "Latitude", "Relocation rate",],
    ),
    ("Optimized", "Relocation rate"): (
        "Optimized livability",
        ["Name", "Longitude", "Latitude", "Relocation rate",],
    ),
}

####-----------------------------------------

# Traffic variables
traffic_variables = ["Mobility flow",
                     "Speed",
                     "Travel time",
                     "Lost time",
                     "Noise"]
traffic_units = {"Mobility flow": "# of passengers",
                "Speed": "m/s",
                "Travel time": "# of trips",
                "Lost time": "# of trips",
                "Noise": "dB"}

####-----------------------------------------

# AQ variables
# TODO: replace thresholds with PM2.5 cut-offs
aqi_pm25_thresholds = [0, 10, 25, 50, 75, np.inf]
aqi_labels = ["good", "satisfactory", "fair", "poor", "very poor"]
aq_variables = ["Carbon monoxide",
                "Carbon dioxide",
                "Hydrocarbon",
                "Nitrogen oxides",
                "Respirable particles",
                "Fine particles"]
aq_units = {
"Carbon monoxide": "mg",
"Carbon dioxide": "mg",
"Hydrocarbon": "mg",
"Nitrogen oxides": "mg",
"Respirable particles": "µg",
"Fine particles": "µg"
}

####-----------------------------------------

# Livability variables

####-----------------------------------------

# A function to re-index timesteps from simulation seconds to minutes
def new_timeline(network, timestep_range):
    timesteps = np.arange(network["Simulation timestep"].min(), network["Simulation timestep"].max(), timestep_range*60, dtype=int)
    timestep_labels = timesteps//60
    network["Timestep"] = pd.cut(network["Simulation timestep"], 
                                bins=timesteps,
                                labels=timestep_labels[1:],
                                right=False, 
                                include_lowest=True)
    return network

# A function to calculate the bounds for a heatmap
def map_bounds(network):
    west_bound = network["Longitude"].min()
    east_bound = network["Longitude"].max()
    south_bound = network["Latitude"].min()
    north_bound = network["Latitude"].max()
    map_bounds = {
        "west": west_bound * (1 - 2e-4),
        "east": east_bound * (1 + 2e-4),
        "south": south_bound * (1 - 5e-6),
        "north": north_bound * (1 + 5e-6),
    }
    return map_bounds

# A function to set the location text and spatial scope for the data when the location is changed
def spatial_scope(spatial_click_data, network):
    if spatial_click_data is None:
        location = "Location: Network"
    else:
        road_lon = spatial_click_data["points"][0]["lon"]
        road_lat = spatial_click_data["points"][0]["lat"]
        name = spatial_click_data["points"][0]["customdata"][0]
        location = f"Location: {name} ({road_lat:.2f} °N, {road_lon:.2f} °E)"
        edge = spatial_click_data["points"][0]["customdata"][1]
        network = network[network["Edge"] == edge]
    return network, location

# A function to slice and copy the required data from the original dataset
def copy_data(situation, variable):
    dataset_name, required_columns = from_situ_var_to_data_cols[(situation, variable)]
    data = datasets[dataset_name]
    network = data[required_columns].copy()
    return network

# A function to normalize to an array to a range (x,y)
def normalize_range(array, x, y):
    m = min(array)
    range = max(array) - m
    array = (array - m)/range
    range2 = y - x
    normalized = (array*range2) + x
    return list(normalized)

####-----------------------------------------

# App

# Style
external_stylesheets=[dbc.themes.BOOTSTRAP, dbc.icons.BOOTSTRAP]

# Hover layout
hovers=dict(bgcolor="white", font_size=16)

# Main app
app = Dash(__name__,
           external_stylesheets=external_stylesheets,
           suppress_callback_exceptions=True)

# Main layout
app.layout = html.Div(
    [
        # Page title
        dbc.Row(
            [
                dbc.Col(
                    html.H1(
                        "AI-based optimisation tool for sustainable urban planning (AIOSut)",
                    ),
                    width="auto",
                ),
                dbc.Col(
                    # More details about the project
                    dbc.Button(
                        "Say what?",
                        id="project-info-button",
                        className="mb-3",
                        color="primary",
                        n_clicks=0,
                        style={"float": "right"},
                    ),
                ),
            ],
            align="center",
            style={"padding-top": "2vh", "padding-bottom": "2vh"},
        ),
        html.Div(
            [
                dbc.Collapse(
                    dbc.Card(
                        dbc.CardBody(
                            dcc.Markdown(
                                """
                                         **AIOSut** is a Proof of Concept -project funded by the Research Council of Finland. The project's aim is to develop a prototype tool that utilizes AI-based scalable and generalisable models for urban planning optimization and features a visual interface that makes the use of the models and analysing of the results easy, without requiring AI knowledge from the user.
                                         
                                         [GitHub page](https://github.com/helsinki-sda-group/dev_aiosut_ui)
                                         """,
                                link_target="_blank",
                            ),
                        ),
                    ),
                    id="project-info-collapse",
                    is_open=False,
                    style={"width": "95vw"},
                ),
            ],
            style={"padding-bottom": "2vh", "padding-top": "2vh"},
        ),
        # Dividing line between the header and interfaces
        html.Hr(),
        # Scenario params
        html.Div(
            [
                dbc.Row(
                    [
                        dbc.Col(
                            # Title
                            html.H2(
                                "Scenario parameters", style={"display": "inline-block"}
                            ),
                        ),
                        dbc.Col(
                            # More details about the project
                            dbc.Button(
                                "What do these do?",
                                id="scenario-info-button",
                                className="mb-3",
                                color="primary",
                                n_clicks=0,
                                style={"float": "right"},
                            ),
                        ),
                    ],
                    align="center",
                    style={"padding-top": "2vh", "padding-bottom": "2vh"},
                )
            ]
        ),
        html.Div(
            [
                dbc.Collapse(
                    dbc.Card(
                        dbc.CardBody(
                            dcc.Markdown(
                                """
                                These are the tools to make your very own simulated city ("scenario" for short)!

                                * *Season* controls the weather conditions - keeping the winters cool and summers hot! Bikers love it.
                                * *Time of week* refers to what day should be simulated. *Weekday* means it is time to hurry to work between Monday and Friday and *Weekend* is for that relaxed sightseeing on Saturday or Sunday.
                                * *Demand* decides how much mobility is going on. *Regular* projection refers to what one would expect to see based on the historical data; *Low* scales the regular projection down by 20%, while *High* increases it by 20%.
                                * *Area* changes what area is being simulated. Trying to take it all in all at once made our head spin (the AI's too), so we made a tour around the city instead.
                                
                                """,
                            ),
                        ),
                    ),
                    id="scenario-info-collapse",
                    is_open=False,
                    style={"width": "95vw"},
                ),
            ],
            style={"padding-top": "2vh", "padding-bottom": "2vh"},
        ),
        dbc.Row(
            [
                dbc.Col(
                    # Buttons for season
                    html.Div(
                        [
                            html.Label(
                                "Season",
                                style={
                                    "font-size": 24,
                                    "padding-bottom": "2vh",
                                    "padding-top": "1vh",
                                },
                            ),
                            dcc.Dropdown(
                                options=seasons,
                                value="Summer",
                                id="crossfilter-season",
                                style={
                                    "font-size": 18,
                                    "width": "95%",
                                },
                            ),
                        ],
                        style={"padding-top": "2vh", "padding-bottom": "2vh"},
                    ),
                ),
                dbc.Col(
                    # Dropdown for weekday
                    html.Div(
                        [
                            html.Label(
                                "Time of week",
                                style={
                                    "font-size": 24,
                                    "padding-bottom": "2vh",
                                    "padding-top": "1vh",
                                },
                            ),
                            dcc.Dropdown(
                                options=weekdays,
                                value="Weekday",
                                id="crossfilter-day",
                                style={
                                    "font-size": 18,
                                    "width": "95%",
                                },
                            ),
                        ],
                        style={"padding-top": "2vh", "padding-bottom": "2vh"},
                    ),
                ),
                dbc.Col(
                    # Buttons for traffic demand
                    html.Div(
                        [
                            html.Label(
                                "Demand",
                                style={
                                    "font-size": 24,
                                    "padding-bottom": "2vh",
                                    "padding-top": "1vh",
                                },
                            ),
                            dcc.Dropdown(
                                options=mobility_demand,
                                value="Regular",
                                id="crossfilter-mobility-demand",
                                style={
                                    "font-size": 18,
                                    "width": "95%",
                                },
                            ),
                        ],
                        style={"padding-top": "2vh", "padding-bottom": "2vh"},
                    ),
                ),
                dbc.Col(
                    # Dropdown for area
                    html.Div(
                        [
                            html.Label(
                                "Area",
                                style={
                                    "font-size": 24,
                                    "padding-bottom": "2vh",
                                    "padding-top": "1vh",
                                },
                            ),
                            dcc.Dropdown(
                                options=areas,
                                value="Kamppi",
                                id="crossfilter-area",
                                style={
                                    "font-size": 18,
                                    "width": "95%",
                                },
                            ),
                        ],
                        style={"padding-top": "2vh", "padding-bottom": "2vh"},
                    ),
                ),
            ],
        ),
        # Dividing line between the parameters
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(
                    # Title
                    html.H2("Optimization parameters"),
                ),
                dbc.Col(
                    # More details about the project
                    dbc.Button(
                        "Now you lost me!",
                        id="optimization-info-button",
                        className="mb-3",
                        color="primary",
                        n_clicks=0,
                        style={"float": "right"},
                    ),
                ),
            ],
            align="center",
            style={"padding-top": "2vh", "padding-bottom": "2vh"},
        ),
        html.Div(
            [
                dbc.Collapse(
                    dbc.Card(
                        dbc.CardBody(
                            dcc.Markdown(
                                """
                                Tuning these parameters to your liking before pressing *Simulate* makes the AI do your bidding in the city renovation!

                                * *Priority* refers to how important the objective is for you. When the AI runs into a tie, the most important objective wins one round of rock-paper-scissors and the game continues.
                                * *Performance thresholds* - because sometimes, one doesn't need to go all the way and instead just do good enough! By decreasing the threshold, you are telling AI "it is okay to rest when you reach _this much_ improvement for a given objective, with respect to the baseline". As a bonus, the results are returned quicker! When set at *100*, the AI returns its ultimate best effort at the solution.
                                
                                """,
                            ),
                        ),
                    ),
                    id="optimization-info-collapse",
                    is_open=False,
                    style={"width": "95vw"},
                ),
            ],
            style={"padding-top": "2vh", "padding-bottom": "2vh"},
        ),
        # Optimization params
        html.Div(
            [
                dbc.Row(
                    [
                        dbc.Col(
                            html.Label(
                                "Objective",
                            ),
                            width={"size": 2},
                        ),
                        dbc.Col(
                            html.Center(
                                html.Label(
                                    "Priority",
                                ),
                            ),
                        ),
                        dbc.Col(
                            html.Center(
                                html.Label(
                                    "Performance thresholds",
                                ),
                            ),
                        ),
                    ],
                    style={
                        "padding-bottom": "2vh",
                        "padding-top": "1vh",
                        "font-size": 24,
                    },
                ),
                dbc.Row(
                    [
                        dbc.Col(
                            html.Label(
                                "Traffic",
                                style={
                                    "font-size": 18,
                                },
                            ),
                            width={"size": 2},
                        ),
                        dbc.Col(
                            dcc.Slider(
                                min=0,
                                max=100,
                                step=10,
                                value=50,
                                id="traffic-optim-slider",
                            ),
                            align="end",
                        ),
                        dbc.Col(
                            html.Center(
                                html.Div(
                                    [
                                        daq.NumericInput(
                                            value=100,
                                            id="crossfilter-traffic-optim-threshold",
                                            min=0,
                                            max=100,
                                            label="% of improvement",
                                            labelPosition="bottom",
                                            style={
                                                "display": "inline-block",
                                                "font-size": 18,
                                                "padding-right": "1vw",
                                            },
                                        ),
                                    ]
                                ),
                            ),
                        ),
                    ],
                    style={
                        "font-size": 18,
                        "padding-bottom": "2vh",
                        "padding-top": "2vh",
                    },
                    align="center",
                ),
                dbc.Row(
                    [
                        dbc.Col(
                            html.Label(
                                "Air quality",
                                style={
                                    "font-size": 18,
                                },
                            ),
                            width={"size": 2},
                        ),
                        dbc.Col(
                            dcc.Slider(
                                min=0,
                                max=100,
                                step=10,
                                value=50,
                                id="air-quality-optim-slider",
                            ),
                            align="end",
                        ),
                        dbc.Col(
                            html.Center(
                                html.Div(
                                    [
                                        daq.NumericInput(
                                            value=100,
                                            id="crossfilter-aq-optim-threshold",
                                            min=0,
                                            max=100,
                                            label="% of improvement",
                                            labelPosition="bottom",
                                            style={
                                                "display": "inline-block",
                                                "font-size": 18,
                                                "padding-right": "1vw",
                                            },
                                        ),
                                    ]
                                ),
                            ),
                        ),
                    ],
                    style={
                        "font-size": 18,
                        "padding-bottom": "2vh",
                        "padding-top": "2vh",
                    },
                    align="center",
                ),
                dbc.Row(
                    [
                        dbc.Col(
                            html.Label(
                                "Livability",
                                style={
                                    "font-size": 18,
                                },
                            ),
                            width={"size": 2},
                        ),
                        dbc.Col(
                            dcc.Slider(
                                min=0,
                                max=100,
                                step=10,
                                value=50,
                                id="livability-optim-slider",
                            ),
                            align="end",
                        ),
                        dbc.Col(
                            html.Center(
                                html.Div(
                                    [
                                        daq.NumericInput(
                                            value=100,
                                            id="crossfilter-liv-optim-threshold",
                                            label="% of improvement",
                                            labelPosition="bottom",
                                            min=0,
                                            max=100,
                                            style={
                                                "display": "inline-block",
                                                "font-size": 18,
                                                "padding-right": "1vw",
                                            },
                                        ),
                                    ]
                                ),
                            ),
                        ),
                    ],
                    style={
                        "font-size": 18,
                        "padding-bottom": "1vh",
                        "padding-top": "1vh",
                    },
                    align="center",
                ),
            ]
        ),
        # Simulate button
        html.Div(
            html.Center(
                dbc.Button(
                    html.Label(
                        "Simulate",
                        style={
                            "font-size": 18,
                        },
                    ),
                    size="lg",
                ),
                style={
                    "padding-top": "2vh",
                    "padding-bottom": "2vh",
                    "display": "block",
                },
            ),
            style={"padding-top": "3vh"},
        ),
        # Dividing line between the parameters
        html.Hr(),
        # Viz params
        dbc.Row(
            [
                dbc.Col(
                    # Title
                    html.H2(
                        "Visualization parameters",
                    ),
                ),
                dbc.Col(
                    # More details about the project
                    dbc.Button(
                        "Come again?",
                        id="visualization-info-button",
                        className="mb-3",
                        color="primary",
                        n_clicks=0,
                        style={"float": "right"},
                    ),
                ),
            ],
            align="center",
            style={"padding-top": "2vh", "padding-bottom": "2vh"},
        ),
        html.Div(
            [
                dbc.Collapse(
                    dbc.Card(
                        dbc.CardBody(
                            dcc.Markdown(
                                """
                                When you get your BIG dataset back, not everything needs to be there, right? Enter filters!

                                * *Temporal aggregation* defines how seconds are aggregated inside a given time range. "Sum" just sums everything&ast; together, while "Average" takes the mean of everything&ast;. For example: "Sum" `[1,2,3] -> 6`, while "Average" `[1,2,3] -> 2`.
                                * *Timestep interval* adjusts how much time you really need to see in the results. As a default, we were thinking in minutes, but you are free to think as long as the simulation lasts. For example, "1" minute makes up for 60 data points out of 60 minutes, while "15" makes up for four and "60" makes up for one.

                                &ast;Well, everything except for Speed and Noise as we can't provide speed of light -travelling or hearing aids here, sorry! Instead, Speed gets averaged always while Noise has its own formula called Harmonoise when it is not averaged.
                                
                                """,
                            ),
                        ),
                    ),
                    id="visualization-info-collapse",
                    is_open=False,
                    style={"width": "95vw"},
                ),
            ],
            style={"padding-top": "2vh", "padding-bottom": "2vh"},
        ),
        # Toggles and texts for parameters
        dbc.Row(
            [
                # Timeline type column
                dbc.Col(
                    html.Div(
                        [
                            html.Label(
                                "Temporal aggregation", style={"padding-bottom": "2vh"}
                            ),
                            dcc.RadioItems(
                                options=["Sum", "Average"],
                                value="Sum",
                                id="crossfilter-timeline-type",
                                inline=True,
                                labelStyle={
                                    "automargin": True,
                                    "padding-top": "1vh",
                                    "font-size": 18,
                                    "padding-right": "1vw",
                                },
                                inputStyle={"margin-right": "0.3vw"},
                            ),
                        ]
                    )
                ),
                # Timestep interval column
                dbc.Col(
                    html.Div(
                        [
                            html.Label(
                                "Timestep interval", style={"padding-bottom": "2vh"}
                            ),
                            html.Div(
                                [
                                    daq.NumericInput(
                                        value=1,
                                        id="crossfilter-timestep-range",
                                        label="minute(s)",
                                        labelPosition="bottom",
                                        min=1,
                                        max=timeline.max(),
                                        style={
                                            "display": "inline-block",
                                            "font-size": 18,
                                            "padding-right": "1vw",
                                        },
                                    ),
                                ]
                            ),
                        ]
                    )
                ),
                # Row end
            ],
            justify="center",
            style={"automargin": True, "padding-top": "2vh", "font-size": 24},
        ),
        # Dividing line between the params and plots
        html.Hr(),
        # Title
        dbc.Row(
            [
                dbc.Col(html.H2("Results")),
                dbc.Col(
                    # More details about the project
                    dbc.Button(
                        "Tell me more, please!",
                        id="results-info-button",
                        className="mb-3",
                        color="primary",
                        n_clicks=0,
                        style={"float": "right"},
                    ),
                ),
            ],
            align="center",
            style={"padding-top": "2vh", "padding-bottom": "2vh"},
        ),
        html.Div(
            [
                dbc.Collapse(
                    dbc.Card(
                        dbc.CardBody(
                            dcc.Markdown(
                                """
                                There is a lot going on in the city - a bit too much, which is why we brought you tabs. *Summary* lets you see the big picture "before" (*Baseline*) and "after" (*Optimized*) the AI renovation, re-counting AI's side of the story, while the other tabs – *Traffic*, *Air quality* and *Livability* – zoom in one objective and different variables related to that objective.

                                FYI: *Baseline* corresponds to the city as of now, using various data recorded all over the city since 2018. That's how our simulation gets real!
                                """,
                            ),
                        ),
                    ),
                    id="results-info-collapse",
                    is_open=False,
                    style={"width": "95vw"},
                ),
            ],
            style={"padding-top": "2vh", "padding-bottom": "2vh"},
        ),
        # Tabs
        dcc.Tabs(
            id="tabs-1",
            value="traffic",
            children=[
                dcc.Tab(label="Summary", value="summary"),
                dcc.Tab(label="Traffic", value="traffic"),
                dcc.Tab(label="Air quality", value="air-quality"),
                dcc.Tab(label="Livability", value="livability"),
            ],
        ),
        html.Div(id="tabs-content-1", style={"padding-top": "1vh"}),
        # Main layout style
    ],
    style={"width": "99vw", "automargin": True, "padding": "5vh 5vh 5vh 5vh"},
)

####-----------------------------------------

### Callbacks

## Tab buttons' callback
@app.callback(
    Output('tabs-content-1', 'children'),
    Input('tabs-1', 'value')
)
def renderViewContent(tab):

    ####-----------------------------------------
    # View for summary tab
    if tab == 'summary':
        return html.Label("Summary under work", style={"padding-top": "2vh", "font-size": 20})

    ####-----------------------------------------
    # View for traffic tab
    elif tab == 'traffic':
        return html.Center(
            [
            # Current or optimized situation
            html.Div(
                [
                    html.Label(
                        "Situation",
                        style={
                            "font-size": 24,
                            "padding-bottom": "2vh",
                            "padding-top": "1vh",
                        },
                    ),
                    dcc.RadioItems(
                        options=["Baseline", "Optimized"],
                        value="Baseline",
                        id="crossfilter-situation-traffic",
                        inline=True,
                        labelStyle={
                            "automargin": True,
                            "padding-top": "1vh",
                            "font-size": 18,
                            "padding-right": "1vw",
                        },
                        inputStyle={"margin-right": "0.4vw"},
                    ),
                ],
                style={"padding-top": "2vh", "padding-bottom": "2vh"},
            ),
              
            # Variable dropdown
            html.Div(
                [
                html.Label(
                    "Variable",
                    style={
                        "font-size": 24,
                        "padding-bottom": "2vh",
                        "padding-top": "1vh",
                    },
                ),
                    dcc.Dropdown(
                        options=traffic_variables,
                        value="Mobility flow",
                        id="crossfilter-variable-traffic",
                        style={
                            "font-size": 18,
                            "width": "50%",
                            "margin": "auto",
                            "text-align": "center",
                        },
                    ),
                ],
                style={"padding-top": "2vh", "padding-bottom": "2vh"},
            ),

            # Heatmap
            html.Div(
                [
                    dcc.Loading(
                        dcc.Graph(
                            id="traffic-one",
                            responsive=True,
                            style={
                                "height": "45vw",
                                "width": "68vw",
                            },
                            config={"showTips": True},
                        ),
                        type="cube",
                    ),
                    # Title
                    html.Label(
                        html.Pre(id="location-text-traffic"),
                        style={
                            "font-size": 24,
                            "padding-bottom": "4vh",
                            "display": "block",
                            "padding-top": "2vh",
                        },
                    ),
                    # Reset button
                    dbc.Button(
                        "Reset location",
                        id="reset-button-traffic",
                        n_clicks=0,
                        style={
                            "display": "block",
                            "font-size": 18,
                            "automargin": True,
                            "padding": "1vh",
                        },
                    ),
                ],
                id = "traffic-one-div",
            ),

            # Bar plot
            html.Div(
                dcc.Loading(
                    dcc.Graph(
                        id="traffic-two",
                        responsive=True,
                        style={"height": "30vw"},
                    ),
                    type="cube",
                ),
                id = "traffic-two-div",
            ),

            # Area chart
            html.Div(
                    dcc.Loading(
                        dcc.Graph(
                            id="traffic-three",
                            responsive=True,
                            style={"height": "30vw"},
                        ),
                        type="cube",
                    ),
                id = "traffic-three-div",
            ),
        ]
    ),

    ####-----------------------------------------
    # View for AQ tab
    elif tab == 'air-quality':
        return html.Center(
            [
            # Current or optimized situation
            html.Div(
                [
                    html.Label(
                        "Situation",
                        style={
                            "font-size": 24,
                            "padding-bottom": "2vh",
                            "padding-top": "1vh",
                        },
                    ),
                    dcc.RadioItems(
                        options=["Baseline", "Optimized"],
                        value="Baseline",
                        id="crossfilter-situation-aq",
                        inline=True,
                        labelStyle={
                            "automargin": True,
                            "padding-top": "1vh",
                            "font-size": 18,
                            "padding-right": "1vw",
                        },
                        inputStyle={"margin-right": "0.4vw"},
                    ),
                ],
                style={"padding-top": "2vh", "padding-bottom": "2vh"},
            ),
            # Variable dropdown
            html.Div(
                [
                    html.Label(
                        "Variable",
                        style={
                            "font-size": 24,
                            "padding-bottom": "2vh",
                            "padding-top": "1vh",
                        },
                    ),
                    dcc.Dropdown(
                        options=aq_variables,
                        value="Carbon monoxide",
                        id="crossfilter-variable-aq",
                        style={
                            "font-size": 18,
                            "width": "50%",
                            "margin": "auto",
                            "text-align": "center",
                        },
                    ),
                ],
                style={"padding-top": "2vh", "padding-bottom": "2vh"},
            ),
            # Heatmap
            html.Div(
                [
                    # The heatmap
                    dcc.Loading(
                        dcc.Graph(
                            id="aq-one",
                            responsive=True,
                            config={"showTips": True},
                            style={"height": "45vw", "width": "68vw"},
                        ),
                        type="cube",
                    ),
                    # Title
                    html.Label(
                        html.Pre(id="location-text-aq"),
                        style={
                            "font-size": 24,
                            "padding-bottom": "4vh",
                            "display": "block",
                            "padding-top": "2vh",
                        },
                    ),
                    # Reset button
                    dbc.Button(
                        "Reset location",
                        id="reset-button-aq",
                        n_clicks=0,
                        style={
                            "display": "block",
                            "font-size": 18,
                            "automargin": True,
                            "padding": "1vh",
                        },
                    ),
                ],
                id = "aq-one-div",
            ),
            # Bar plot
            html.Div(     
                dcc.Loading(
                    dcc.Graph(
                        id="aq-two",
                        responsive=True,
                        config={"showTips": True},
                        style={"height": "30vw"},
                    ),
                    type="cube",
                ),
                id = "aq-two-div",
            ),
            # Area chart
            html.Div(
                dcc.Loading(
                    dcc.Graph(
                        id="aq-three",
                        responsive=True,
                        style={"height": "30vw"},
                    ),
                    type="cube",
                ),
                id = "aq-three-div"
            ),
        ],
    ),

    ####-----------------------------------------
    # View for livability tab
    elif tab == 'livability':

        return html.Center(
            [
            # Current or optimized situation
            html.Div([
                html.Label("Situation", style={"font-size": 24, "padding-bottom": "2vh", "padding-top": "1vh"}),
                dcc.RadioItems(options=["Baseline", "Optimized"],
                    value="Baseline",
                    id="crossfilter-situation",
                    inline=True, 
                    labelStyle={"automargin": True, "padding-top": "1vh", "font-size": 18, "padding-right": "1vw"}, 
                    inputStyle={"margin-right": "0.4vw"}),                
                ],
                style={"padding-top": "2vh", "padding-bottom": "2vh"}
            ),
                
            html.Label("Livability under work", style={"padding-top": "2vh", "font-size": 20}),

            ]
        )

####-----------------------------------------

## Simulation button callback

####-----------------------------------------

## Help buttons' callbacks

# Project info help button
@app.callback(
    Output("project-info-collapse", "is_open"),
    [Input("project-info-button", "n_clicks")],
    [State("project-info-collapse", "is_open")],
)
def toggle_project_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

# Scenario parameters' help button
@app.callback(
    Output("scenario-info-collapse", "is_open"),
    [Input("scenario-info-button", "n_clicks")],
    [State("scenario-info-collapse", "is_open")],
)
def toggle_scenario_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

# Optimization parameters' help button
@app.callback(
    Output("optimization-info-collapse", "is_open"),
    [Input("optimization-info-button", "n_clicks")],
    [State("optimization-info-collapse", "is_open")],
)
def toggle_optimization_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

# Visualization parameters' help button
@app.callback(
    Output("visualization-info-collapse", "is_open"),
    [Input("visualization-info-button", "n_clicks")],
    [State("visualization-info-collapse", "is_open")],
)
def toggle_viz_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

# Results' info help button
@app.callback(
    Output("results-info-collapse", "is_open"),
    [Input("results-info-button", "n_clicks")],
    [State("results-info-collapse", "is_open")],
)
def toggle_results_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

####-----------------------------------------

## Summary tab callback

####-----------------------------------------

## Traffic tab callback

@app.callback(
    Output("traffic-one", "figure"),
    Output("traffic-one-div", "style"),
    Output("traffic-two", "figure"),
    Output("traffic-two-div", "style"),
    Output("traffic-three", "figure"),
    Output("traffic-three-div", "style"),
    Output("location-text-traffic", "children"),
    Input("crossfilter-situation-traffic", "value"),
    Input("crossfilter-variable-traffic", "value"),
    Input("crossfilter-timestep-range", "value"),
    Input("crossfilter-timeline-type", "value"),
    Input("traffic-one", "clickData"),
)
def drawTraffic(situation, variable, timestep_range, timeline_type, spatial_click_data):
    # Initialize default outputs
    first_plot = {}
    second_plot = {}
    third_plot = {}
    first_plot_style = {"display": "none"}
    second_plot_style = {"display": "none"}
    third_plot_style = {"display": "none"}
    location = ""

    # Heatmap + area chart
    if variable in ["Mobility flow"]:
        # Make a copy of the required data for all plots (to avoid changing the simulation output)
        network = copy_data(situation=situation, variable=variable)
        network = new_timeline(network=network, timestep_range=timestep_range)
        network, location = spatial_scope(spatial_click_data, network)
        bounds = map_bounds(network)

        # Calculate data
        heatmap_network = (
                network.groupby(["Timestep", "Edge"], observed=False)
                .agg(
                    {
                        variable: timeline_functions[timeline_type],
                        "Longitude": "first",
                        "Latitude": "first",
                        "Name": "first",
                    }
                )
                .reset_index()
            )
        # Customize legend
        legend_max = 1.2 * heatmap_network[heatmap_network[variable] != 0][variable].median()
        # Draw the heatmap
        heatmap = px.density_map(
            data_frame=heatmap_network,
            lat="Latitude",
            lon="Longitude",
            z=variable,
            radius=15,
            map_style="open-street-map",
            range_color=(0, legend_max),
            hover_data={
                "Name": True,
                "Edge": False,
                "Mobility flow": True,
                "Longitude": False,
                "Latitude": False,
                variable: True,
            },
            animation_frame="Timestep",
            title="Network heatmap",
        )
        # Customize the sliders with a larger font size and prefix
        sliders = [dict(
        currentvalue={"prefix": "Time (in minutes): "},
        font={"size": 14},
        pad={"t": 50}
        )]
        # Update points' opacity on click
        heatmap.update_layout(
            clickmode="event+select",
            sliders=sliders,
            map_bounds=bounds
        )
        # Sets the variable name and its unit as a legend to the color bar
        heatmap.layout["coloraxis"]["colorbar"]["title"] = f"{variable} ({traffic_units[variable]})"
        # Output the plot
        first_plot = heatmap
        first_plot_style = {"padding-top": "3vh"}

        # Calculate data
        area_network = network.groupby(["Mobility mode", "Timestep"], observed=False).agg(
                {variable: timeline_functions[timeline_type]}
                ).reset_index()
        # Draw the area chart
        area_plot = px.area(
                data_frame=area_network,
                x="Timestep",
                y=variable,
                color="Mobility mode",
                color_discrete_sequence=px.colors.sequential.Plasma_r,
                title="Mobility mode time series",
            )
        # Customize the title and yaxis
        area_plot.update_layout(title_x=0.11,
        yaxis_title=f"{variable} ({traffic_units[variable]})")
        # Ooutput the plot
        third_plot = area_plot
        third_plot_style = {"padding-top": "3vh"}

    # Histogram + bar plot
    elif variable in ["Travel time", "Lost time"]:
        # Calculate data
        barplot_network = copy_data(situation=situation, variable=variable)
        # Draw the histogram
        bar_plot = px.histogram(data_frame=barplot_network, x=variable, color="Mobility mode", barmode="overlay")
        # Update the axis labels and the bars' gap
        bar_plot.update_layout(yaxis_title = traffic_units[variable], bargap=0.2, xaxis_title = f"{variable} in seconds")
        # Output the plot
        second_plot = bar_plot
        second_plot_style = {"padding-top": "3vh"}

    # Heatmap + bar plot
    elif variable in ["Noise", "Speed"]:
        # Calculate the data
        network = copy_data(situation=situation, variable=variable)
        network = new_timeline(network=network, timestep_range=timestep_range)
        network, location = spatial_scope(spatial_click_data, network)
        bounds = map_bounds(network)

        # Calculate the data
        heatmap_network = (
                    network.groupby(["Timestep", "Edge"], observed=False)
                    .agg(
                        {
                            variable: timeline_functions[timeline_type],
                            "Longitude": "first",
                            "Latitude": "first",
                            "Name": "first",
                            "Mobility flow": timeline_functions[timeline_type],
                        }
                    )
                    .reset_index()
                )
        # Customize legend
        legend_max = 1.2 * heatmap_network[heatmap_network[variable] != 0][variable].median()
        # Draw the heatmap
        heatmap = px.density_map(data_frame=heatmap_network,
        lat = "Latitude",
        lon = "Longitude",
        z = variable,
        radius=15,
        zoom=10,
        range_color = (0,legend_max),
        map_style="open-street-map",
        hover_data = {"Name": True, 
                    "Edge": False,
                    "Mobility flow": True, 
                    "Longitude": False, 
                    "Latitude": False, 
                    variable: True},
        animation_frame="Timestep",
        title="Network heatmap"
        )
        # Customize the sliders with a larger font size and prefix
        sliders = [dict(
        currentvalue={"prefix": "Time (in minutes): "},
        # active=active_frame,
        font={"size": 14},
        pad={"t": 50}
        )]
        # Update points' opacity on click
        heatmap.update_layout(
            clickmode="event+select",
            sliders=sliders,
            map_bounds=bounds
        )
        # Sets the variable name and its unit as a legend to the color bar
        heatmap.layout["coloraxis"]["colorbar"]["title"] = f"{variable} ({traffic_units[variable]})"
        first_plot = heatmap
        first_plot_style = {"padding-top": "3vh"}

        # Calculate the data
        secondary_network = datasets["Optimized emissions"].copy()
        barplot_network = (
            secondary_network.groupby(["Mobility mode"], observed=False)
            .agg({variable: "sum", "Mobility flow": "sum"})
            .reset_index()
        )
        barplot_network["Vehicle average"] = np.round(barplot_network[variable] / barplot_network["Mobility flow"], 4)
        # Draw the bar plot
        bar_plot = px.bar(
            data_frame=barplot_network,
            x=variable,
            y="Mobility mode",
            color="Vehicle average",
            hover_data="Mobility flow",
            title="Vehicle averages per mobility mode",
        )
        # Customize the hovers, title, axis labels and legend
        bar_plot.update_layout(
            hoverlabel=hovers,
            title_x=0.11,
            bargap=0.5,
            xaxis_title=f"{variable} ({traffic_units[variable]})",
            yaxis_title="Mobility mode",
            xaxis=dict(range=[0, barplot_network[variable].max()*1.1]),
        )
        # Output the plot
        second_plot = bar_plot
        second_plot_style = {"padding-top": "3vh"}

        # Calculate the data
        area_network = (
            network.groupby(["Timestep"], observed=False)
            .agg({variable: timeline_functions[timeline_type], "Mobility flow": timeline_functions[timeline_type]})
            .reset_index()
        )
        # Draw the area chart
        area_plot = px.area(
            data_frame=area_network,
            x="Timestep",
            y=variable,
            color_discrete_sequence=px.colors.sequential.Plasma_r,
            hover_data=["Mobility flow"],
            title="Mobility mode time series",
        )
        # Customize the hovers, title and yaxis
        area_plot.update_layout(
            hoverlabel=hovers,
            title_x=0.11,
            yaxis_title=f"{variable} ({traffic_units[variable]})",
        )
        third_plot = area_plot
        third_plot_style = {"padding-top": "3vh"}

    # Return the figures, styles and location
    return first_plot, first_plot_style, second_plot, second_plot_style, third_plot, third_plot_style, location

# Reset the traffic location
@app.callback(
    Output("traffic-one", "clickData"),
    [Input("reset-button-traffic", "n_clicks")])
def resetTrafficLocation(reset):
    return None

####-----------------------------------------

## Air quality tab callback

@app.callback(
    Output("aq-one", "figure"),
    Output("aq-one", "style"),
    Output("aq-two", "figure"),
    Output("aq-two", "style"),
    Output("aq-three", "figure"),
    Output("aq-three", "style"),
    Output("location-text-aq", "children"),
    Input("crossfilter-situation-aq", "value"),
    Input("crossfilter-variable-aq", "value"),
    Input("crossfilter-timestep-range", "value"),
    Input("crossfilter-timeline-type", "value"),
    Input("aq-one", "clickData"),
)
def drawAirQuality(situation, variable, timestep_range, timeline_type, spatial_click_data):
    # Initialize default outputs
    first_plot = {}
    second_plot = {}
    third_plot = {}
    first_plot_style = {"display": "none"}
    second_plot_style = {"display": "none"}
    third_plot_style = {"display": "none"}
    location = ""

    # Make a copy of the initial data (to avoid changing the simulation output)
    network = copy_data(situation=situation, variable=variable)
    network = new_timeline(network=network, timestep_range=timestep_range)
    network, location = spatial_scope(spatial_click_data, network)
    bounds = map_bounds(network)

    # Heatmap + bar plot + area chart
    # Calculate the data
    heatmap_network = (
                network.groupby(["Timestep", "Edge"], observed=False)
                .agg(
                    {
                        variable: timeline_functions[timeline_type],
                        "Longitude": "first",
                        "Latitude": "first",
                        "Name": "first",
                        "Mobility flow": "sum",
                    }
                )
                .reset_index()
            )
    # Customize legend
    legend_max = 1.2 * heatmap_network[heatmap_network[variable] != 0][variable].median()
    # Draw the heatmap
    heatmap = px.density_map(data_frame=heatmap_network,
    lat = "Latitude",
    lon = "Longitude",
    z = variable,
    radius=15,
    zoom=10,
    range_color = (0,legend_max),
    map_style="open-street-map",
    hover_data = {"Name": True, 
                  "Edge": False,
                  "Mobility flow": True,
                  "Longitude": False, 
                  "Latitude": False, 
                  variable: True},
    animation_frame="Timestep",
    title="Network heatmap"
    )
    # Customize the sliders with a larger font size and prefix
    sliders = [dict(
    currentvalue={"prefix": "Time (in minutes): "},
    font={"size": 14},
    pad={"t": 50}
    )]
    # Update points' opacity on click
    heatmap.update_layout(
        clickmode="event+select",
        sliders=sliders,
        map_bounds=bounds
    )
    # Sets the variable name and its unit as a legend to the color bar
    heatmap.layout["coloraxis"]["colorbar"]["title"] = f"{variable} ({aq_units[variable]})"
    first_plot = heatmap
    first_plot_style = {"padding-top": "3vh"}

    # Calculate the data
    barplot_network = (
        network.groupby(["Mobility mode"], observed=False)
        .agg({variable: "sum", "Mobility flow": "sum"})
        .reset_index()
    )
    barplot_network["Vehicle average"] = np.round(barplot_network[variable] / barplot_network["Mobility flow"], 4)
    # Draw the bar plot
    bar_plot = px.bar(
        data_frame=barplot_network,
        x=variable,
        y="Mobility mode",
        color="Vehicle average",
        hover_data="Mobility flow",
        title="Vehicle averages per mobility mode",
    )
    # Customize the hovers, title, axis labels and legend
    bar_plot.update_layout(
        hoverlabel=hovers,
        title_x=0.11,
        bargap=0.5,
        xaxis_title=f"{variable} ({aq_units[variable]})",
        yaxis_title="Mobility mode",
        xaxis=dict(range=[0, barplot_network[variable].max()]),
    )
    # Output the plot
    second_plot = bar_plot
    second_plot_style = {"padding-top": "3vh"}

    # Calculate the data
    area_network = (
        network.groupby(["Mobility mode", "Timestep"], observed=False)
        .agg({variable: timeline_functions[timeline_type], "Mobility flow": timeline_functions[timeline_type]})
        .reset_index()
    )
    # Draw the area chart
    area_plot = px.area(
        data_frame=area_network,
        x="Timestep",
        y=variable,
        color="Mobility mode",
        color_discrete_sequence=px.colors.sequential.Plasma_r,
        hover_data=["Mobility flow"],
        title="Mobility mode time series",
    )
    # Customize the hovers, title and yaxis
    area_plot.update_layout(
        hoverlabel=hovers,
        title_x=0.11,
        yaxis_title=f"{variable} ({aq_units[variable]})",
    )
    # Output the plot
    third_plot = area_plot
    third_plot_style = {"padding-top": "3vh"}

    # Return the figures, styles and location
    return first_plot, first_plot_style, second_plot, second_plot_style, third_plot, third_plot_style, location

# Reset the AQ location
@app.callback(
    Output("aq-one", "clickData"),
    [Input("reset-button-aq", "n_clicks")])
def resetAirQualityLocation(reset):
    return None

####-----------------------------------------

## Livability tab callback

####-----------------------------------------

# Run the app

app.run(debug=True)